# ADAMTS15 individuals
Variants in ADAMTS15 are associated with [Arthrogryposis, distal, type 12](https://omim.org/entry/620545)

In [2]:
import pandas as pd
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.61


In [3]:
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
created_by="ORCID:0000-0002-0736-9199"
print(f"HPO version {hpo_version}")

HPO version 2024-02-27


In [4]:
df = pd.read_excel("input/ADAMTS15_individuals.xlsx")
df.head(2)

,PMID,title,individual_id,comment,disease_id,disease_label,HGNC_id,gene_symbol,transcript,allele_1,...,High palate,Dental crowding,Agenesis of maxillary incisor,Scoliosis,Spinal rigidity,Thoracic kyphosis,Cryptorchidism,Inguinal hernia,Hydrocele testis,Mitral valve prolapse
0,CURIE,str,str,optional,CURIE,str,CURIE,str,str,str,...,HP:0000218,HP:0000678,HP:0200160,HP:0002650,HP:0003306,HP:0002942,HP:0000028,HP:0000023,HP:0000034,HP:0001634
1,PMID:35962790,Biallelic variants in ADAMTS15 cause a novel form of distal arthrogryposis,Family A Individual 1 (II-1),NaN,OMIM:620545,"Arthrogryposis, distal, type 12",HGNC:16305,ADAMTS15,NaN,c.123C>G,...,na,observed,excluded,observed,observed,observed,observed,excluded,excluded,excluded


In [6]:
encoder = CaseTemplateEncoder(df=df, hpo_cr=hpo_cr, created_by=created_by, hpo_ontology=hpo_ontology)
individuals = encoder.get_individuals()

Created encoders for 43 fields


In [7]:
ADAMTS15_transcript = "NM_139055.4"
vmanager = VariantManager(df=df,
                          individual_column_name="individual_id",
                          gene_symbol="ADAMTS15",
                          transcript=ADAMTS15_transcript,
                          allele_1_column_name="allele_1",
                          allele_2_column_name="allele_2")

In [8]:
vmanager.to_summary()

,status,count,alleles
0,mapped,4,"c.2715C>G, c.123C>G, c.2281G>A, c.1903-2A>G"
1,unmapped,0,


In [9]:
vmanager.add_variants_to_individuals(individuals)

In [10]:
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1,
                                allelic_requirement=AllelicRequirement.BI_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

In [11]:
individuals = cvalidator.get_error_free_individual_list()
table = IndividualTable(individuals)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
Family A Individual 1 (II-1) (MALE; P17Y5M),"Arthrogryposis, distal, type 12 (OMIM:620545)",NM_139055.4:c.123C>G (homozygous),Congenital finger flexion contractures (HP:0005879); Absent distal interphalangeal creases (HP:0001032); Hand muscle atrophy (HP:0009130); Tapered distal phalanges of finger (HP:0009884); Clinodactyly of the 5th finger (HP:0004209); Hypoesthesia (HP:0033748); Talipes equinovarus (HP:0001762); Achilles tendon contracture (HP:0001771); Ankle flexion contracture (HP:0006466); Knee flexion contracture (HP:0006380); Acroparesthesia (HP:0031006); Low-set ears (HP:0000369); Strabismus (HP:0000486); Dental crowding (HP:0000678); Scoliosis (HP:0002650); Spinal rigidity (HP:0003306); Thoracic kyphosis (HP:0002942); Cryptorchidism (HP:0000028); excluded: Intellectual disability (HP:0001249); excluded: Palmar hyperhidrosis (HP:0006089); excluded: Hypertelorism (HP:0000316); excluded: Ptosis (HP:0000508); excluded: Agenesis of maxillary incisor (HP:0200160); excluded: Inguinal hernia (HP:0000023); excluded: Hydrocele testis (HP:0000034); excluded: Mitral valve prolapse (HP:0001634)
Family A Individual 2 (II-4) (MALE; P9Y7M),"Arthrogryposis, distal, type 12 (OMIM:620545)",NM_139055.4:c.123C>G (homozygous),Congenital finger flexion contractures (HP:0005879); Absent distal interphalangeal creases (HP:0001032); Hand muscle atrophy (HP:0009130); Tapered distal phalanges of finger (HP:0009884); Clinodactyly of the 5th finger (HP:0004209); Hypoesthesia (HP:0033748); Palmar hyperhidrosis (HP:0006089); Achilles tendon contracture (HP:0001771); Ankle flexion contracture (HP:0006466); Knee flexion contracture (HP:0006380); Low-set ears (HP:0000369); Dental crowding (HP:0000678); Scoliosis (HP:0002650); Spinal rigidity (HP:0003306); Thoracic kyphosis (HP:0002942); excluded: Intellectual disability (HP:0001249); excluded: Talipes equinovarus (HP:0001762); excluded: Acroparesthesia (HP:0031006); excluded: Hypertelorism (HP:0000316); excluded: Strabismus (HP:0000486); excluded: Ptosis (HP:0000508); excluded: Agenesis of maxillary incisor (HP:0200160); excluded: Cryptorchidism (HP:0000028); excluded: Inguinal hernia (HP:0000023); excluded: Hydrocele testis (HP:0000034); excluded: Mitral valve prolapse (HP:0001634)
Family B Individual 3 (II-6) (FEMALE; P17Y5M),"Arthrogryposis, distal, type 12 (OMIM:620545)",NM_139055.4:c.1903-2A>G (homozygous),Congenital finger flexion contractures (HP:0005879); Absent distal interphalangeal creases (HP:0001032); Hand muscle atrophy (HP:0009130); Tapered distal phalanges of finger (HP:0009884); Clinodactyly of the 5th finger (HP:0004209); Hypoesthesia (HP:0033748); Palmar hyperhidrosis (HP:0006089); Ankle flexion contracture (HP:0006466); Knee flexion contracture (HP:0006380); Low-set ears (HP:0000369); Hypertelorism (HP:0000316); Agenesis of maxillary incisor (HP:0200160); Scoliosis (HP:0002650); Spinal rigidity (HP:0003306); Thoracic kyphosis (HP:0002942); Inguinal hernia (HP:0000023); Mitral valve prolapse (HP:0001634); excluded: Intellectual disability (HP:0001249); excluded: Talipes equinovarus (HP:0001762); excluded: Achilles tendon contracture (HP:0001771); excluded: Strabismus (HP:0000486); excluded: Ptosis (HP:0000508); excluded: Dental crowding (HP:0000678)
Family C Individual 4 (II-1) (MALE; P3Y1M),"Arthrogryposis, distal, type 12 (OMIM:620545)",NM_139055.4:c.2281G>A (homozygous),Congenital finger flexion contractures (HP:0005879); Absent distal interphalangeal creases (HP:0001032); Hand muscle atrophy (HP:0009130); Tapered distal phalanges of finger (HP:0009884); Clinodactyly of the 5th finger (HP:0004209); Achilles tendon contracture (HP:0001771); Low-set ears (HP:0000369); Cryptorchidism (HP:0000028); Inguinal hernia (HP:0000023); excluded: Intellectual disability (HP:0001249); excluded: Hypoesthesia (HP:0033748); excluded: Palmar hyperhidrosis (HP:0006089); excluded: Talipes equinovarus (HP:0001762); excluded: Ankle flexion contracture (

In [12]:
encoder.output_individuals_as_phenopackets(individual_list=individuals)

We output 5 GA4GH phenopackets to the directory phenopackets


# HPOA file

In [13]:
ingestor = PhenopacketIngestor(indir="phenopackets")
ppkt_d = ingestor.get_phenopacket_dictionary()
ppkt_list = list(ppkt_d.values())

[pyphetools] Ingested 5 GA4GH phenopackets.


In [14]:
builder = HpoaTableBuilder(phenopacket_list=ppkt_list)
PMID = "PMID:35962790" # Biallelic variants in ADAMTS15 cause a novel form of distal arthrogryposis 
hpoa_table_creator = builder.autosomal_recessive(PMID).build()

We found a total of 27 unique HPO terms
Extracted disease: Arthrogryposis, distal, type 12 (OMIM:620545)


In [15]:
df = hpoa_table_creator.get_dataframe()
df.head(5)

,#diseaseID,diseaseName,phenotypeID,phenotypeName,onsetID,onsetName,frequency,sex,negation,modifier,description,publication,evidence,biocuration
0,OMIM:620545,"Arthrogryposis, distal, type 12",HP:0005879,Congenital finger flexion contractures,,,5/5,,,,,PMID:35962790,PCS,ORCID:0000-0002-0736-9199
1,OMIM:620545,"Arthrogryposis, distal, type 12",HP:0001032,Absent distal interphalangeal creases,,,5/5,,,,,PMID:35962790,PCS,ORCID:0000-0002-0736-9199
2,OMIM:620545,"Arthrogryposis, distal, type 12",HP:0009130,Hand muscle atrophy,,,5/5,,,,,PMID:35962790,PCS,ORCID:0000-0002-0736-9199
3,OMIM:620545,"Arthrogryposis, distal, type 12",HP:0009884,Tapered distal phalanges of finger,,,5/5,,,,,PMID:35962790,PCS,ORCID:0000-0002-0736-9199
4,OMIM:620545,"Arthrogryposis, distal, type 12",HP:0004209,Clinodactyly of the 5th finger,,,5/5,,,,,PMID:35962790,PCS,ORCID:0000-0002-0736-9199


In [16]:
hpoa_table_creator.write_data_frame()

Wrote HPOA disease file to OMIM-620545.tab
